### Importing liblaries 

In [9]:
import numpy as np 
import json 
import logging
import os
import matplotlib.pyplot as plt


In [10]:
# configure the logger
logging.basicConfig(filename="logs.log",
                    format='%(asctime)s %(message)s',
                    filemode='w')
logger = logging.getLogger();
logger.setLevel(logging.DEBUG);


### Pre defined functions used for our analysis 

In [12]:
# This code takes in a PDF file and return list of all ATOM, Helix Record types 
def getAtomRecord(pdb_lines):
    atom_list= [];
    helix_list =[];
    sheet_list = [];
    link_list = [];
    for line in pdb_lines:
        if(line[0:4] == 'ATOM'):
            atom_list.append(line);
        if(line[0:5] == 'HELIX'):
            helix_list.append(line);
        if(line[0:5] == 'SHEET'):
            sheet_list.append(line);
        if(line[0:4] == 'LINK'):
            link_list.append(line);
    return atom_list,helix_list,sheet_list,link_list ;


In [13]:
# takes the atom records of PDB and returns the helix present in the record 
def getAlphaHelixResidues(atom_lines, helix_records):
    logger.info('Extracting alpha helices !')
 #   print('Extracting alpha helices !');

    helix_dict_list = [];
    for ind_helix in helix_records:
        serNo = ind_helix[7:10]
        code= ind_helix[11:14];
        strating_residue =  ind_helix[21:25];
        ending_residue =  ind_helix[33:37];
        helix_length = ind_helix[71:76];

        helx_lines = [];
        flag =0 ;
        for atom_line in atom_lines : 
            res_number = atom_line[22:26];
            
            if(res_number == strating_residue):
                flag = 1;       

            if(int(res_number) > int(ending_residue) or (int(res_number) < int(strating_residue) and flag == 1 )):
                break; 

            if(flag):
                helx_lines.append(atom_line);
    

        #create a list of dictionaries to store each alpha helix information
        helix_dict = {'strating_residue_number':strating_residue ,'ending_residue_number':ending_residue,'length':helix_length ,
                      'helix_S.No':serNo, 'helix_code': code,'helix_residues': helx_lines} ;
        helix_dict_list.append(helix_dict);
    return helix_dict_list;

    

In [14]:
#This takes in a list of atom records and removes all the atmos which are not present in the main chain 
def preserveMainChain(pdb_lines) :
    mainChain_lines = [];
    prev_res = '';
    count =0; 
    for ind_residue in pdb_lines:
        residue_number = ind_residue[22:26];
        if(residue_number != prev_res ):
            count = 1; 
            prev_res= residue_number;

        if(count > 4):
            continue;
        else:
            mainChain_lines.append(ind_residue);
            count = count + 1; 
            
    return mainChain_lines;

In [15]:
# Functino which takes in the main chain information for an alpha helix and calculates the distance bwteen n C and n+1 N atoms 
# alpha helix is a dictionary as described above which contains information about the helix 
def getHBondEuclideanDistanceHelix(alpha_helix):
    helix_length = int(alpha_helix['length']);
    helix_residues = alpha_helix['mainChainHelix_residues'];
    n_nplusfour_distance_array = [];
    broken_H_bonds_array= [];
    if(helix_length < 5 ):
       # print('No H bonding can occur as residue length is ', helix_length);
        alpha_helix['n_nplusfour_distances'] = [];
        alpha_helix['broken_H_bonds']= [];
    else : 
        #print('Helix Lengh is ',helix_length);

        for res_no in range(0 , helix_length - 4):  # we loop from the N terminal to the C terminal of the helix and find distance b/w donor and acceptors 
            donor_atom = res_no*4 + 16;  # stores the index of nitogen atom which gives the H bond
            acceptor_atom = res_no*4 +3 ; # stores the index of the O aton which accepts the hydrogen bond 
            #print('donor atom is' , helix_residues[donor_atom]);
            #print('accpetor atom is' , helix_residues[acceptor_atom]);
            donor_coordinates_x =float(helix_residues[donor_atom][30:38]);
            acceptor_coordinates_x = float(helix_residues[acceptor_atom][30:38]);
            donor_coordinates_y = float(helix_residues[donor_atom][38:46]);
            acceptor_coordinates_y = float(helix_residues[acceptor_atom][38:46]);
            donor_coordinates_z = float(helix_residues[donor_atom][46:54]);
            acceptor_coordinates_z = float(helix_residues[acceptor_atom][46:54]);
            acceptor_point = np.array((donor_coordinates_x, donor_coordinates_y, donor_coordinates_z));
            donor_point =  np.array((acceptor_coordinates_x, acceptor_coordinates_y, acceptor_coordinates_z));
            dist = np.linalg.norm(acceptor_point - donor_point);  # calculates the distance between the donor and acceptor atoms 
            n_nplusfour_distance_array.append(dist);
            if(dist > 3.5 ):
                broken_H_bonds_array.append(res_no);
                
            #print('Distance B/w acceptor and donor is', dist); 
        alpha_helix['n_nplusfour_distances'] = n_nplusfour_distance_array;
        alpha_helix['broken_H_bonds']= broken_H_bonds_array;
    return alpha_helix;

In [16]:
def get_dihedral(p):
    """Praxeolitic formula
    1 sqrt, 1 cross product"""
    p0 = p[0]
    p1 = p[1]
    p2 = p[2]
    p3 = p[3]

    b0 = -1.0*(p1 - p0) 
    b1 = p2 - p1
    b2 = p3 - p2

    # normalize b1 so that it does not influence magnitude of vector
    # rejections that come next
    b1 /= np.linalg.norm(b1)

    # vector rejections
    # v = projection of b0 onto plane perpendicular to b1
    #   = b0 minus component that aligns with b1
    # w = projection of b2 onto plane perpendicular to b1
    #   = b2 minus component that aligns with b1
    v = b0 - np.dot(b0, b1)*b1
    w = b2 - np.dot(b2, b1)*b1

    # angle between v and w in a plane is the torsion angle
    # v and w may not be normalized but that's fine since tan is y/x
    x = np.dot(v, w)
    y = np.dot(np.cross(b1, v), w)
    return np.degrees(np.arctan2(y, x))

In [17]:
def getCoordinateList(atom_record):
    #returns the X, Y, Z coordinates in the form of a list 
    xCoord = float(atom_record[30:38]);
    yCoord = float(atom_record[38:46]);
    zCoord = float(atom_record[46:54]);
    return [xCoord, yCoord, zCoord];

In [18]:
def getDihedralAnglesHelix(alpha_helix):
    helix_length = int(alpha_helix['length']);
    helix_residues = alpha_helix['mainChainHelix_residues'];
    psi_angle_list = [];
    phi_angle_list = [];
    omege_angle_list = [];
    
    for res_no in range(0, helix_length-1):
        #for psi angle 
        first_nitogen = np.array(getCoordinateList(helix_residues[res_no*4])); 
        calpha = np.array(getCoordinateList(helix_residues[(res_no* 4) + 1])); 
        second_calpha = np.array(getCoordinateList(helix_residues[((res_no+1)* 4) + 1])); 
        carbon = np.array(getCoordinateList(helix_residues[(res_no * 4) +2]));
        second_nitrogen = np.array(getCoordinateList(helix_residues[(res_no+1)*4]));
        second_carbon = np.array(getCoordinateList(helix_residues[((res_no +1)* 4) +2]));
        #print( helix_residues[res_no*4],helix_residues[(res_no* 4) + 1],helix_residues[(res_no * 4) +2],helix_residues[(res_no+1)*4])
        psi_angle= get_dihedral(np.array([ first_nitogen,calpha, carbon, second_nitrogen]));
        phi_angle= get_dihedral(np.array([ carbon,second_nitrogen, second_calpha, second_carbon]));
        phi_angle_list.append(phi_angle);
        psi_angle_list.append(psi_angle);
    alpha_helix['psi_angles'] = psi_angle_list;
    alpha_helix['phi_angles'] = phi_angle_list;
    return alpha_helix;
    

In [19]:
def writeHelixFile(helix_dict_list,pdb_code):
        # write the results of analysis in a JSON file
        optuput_file = '../Outputs/brokenHelix/' + pdb_code[:-4] + '.json';

        logger.info('Ssaving Json File');

        #print('Ssaving Json File')

        with open(optuput_file, "w") as output:
            output.write('[');
            for dic in helix_dict_list:
                json.dump(dic,  output, indent = 4);
                output.write(',');

            output.write('{}]');
        logger.info('Success');
        #print('Success');
        return helix_dict_list;

In [45]:
def plotRamachandranAngles(helix_dict_list, pdb_code):
    x=[];
    y=[];
    for ind in helix_dict_list: 
         x= x + (ind['psi_angles']);
         y= y + (ind['phi_angles']);
        
    plt.scatter(x, y);
    plt.xlabel('psi');
    plt.ylabel('phi');
    plt.savefig('../Outputs/brokenHelix/Figures/'+pdb_code+'_ramachandran.png')
    plt.close()





### Analysis Codes

In [35]:
def runBrokenHelixAnalysis(pdb_lines_complete,pdb_code):
    try:
        logger.info('Running broken chain analysis for '+ pdb_code);

        atom_list,helix_list,sheet_list,link_list  = getAtomRecord(pdb_lines_complete);
        print('Number of atoms records present in PDB are', len(atom_list));
        print('Number of  helix present in PDB are', len(helix_list));
        print('Number of Sheets present in PDB are', len(sheet_list));
        print('Number of links present in PDB are', len(link_list));
        # this will give us all the helix present in the protein 

        # Extraxt all the alpha helix from the PDB files and remove all side chain atoms 
        helix_dict_list = getAlphaHelixResidues(atom_list, helix_list);
        logger.info('Getting The main chain residues !');
        #print('Getting The main chain residues !')

        for ind in range(0,len(helix_dict_list)):
            ind_helix = helix_dict_list[ind];
            helixResidue = ind_helix['helix_residues'];
            mainChainResidue = preserveMainChain(helixResidue);
            ind_helix['mainChainHelix_residues']= mainChainResidue;
            helix_dict_list[ind] = ind_helix;
        helix_dict_list

        # for each helix verify if the H bond criteria is fullfilled 
        logger.info('Checking for H bond criteria !');
        #print('Checking for H bond criteria !');

        for ind in range(0,len(helix_dict_list)) : 
            alpha_helix = getHBondEuclideanDistanceHelix(helix_dict_list[ind]);
            helix_dict_list[ind] = alpha_helix;
            
        logger.info('Broken Helix ran sucessfully');

        return helix_dict_list;
    except:
        logger.error('Something Bad hapenned in broken heliz analysis');
        return [];
        #print('Something Bad hapenned');

In [39]:
def getDihedralAngles(helix_dict_list,pdb_code):
    try: 
        logger.info("Performing dihedral angles analysis for " +  pdb_code);
        for ind in range(0,len(helix_dict_list)) : 
            helix_dict_list[ind] = getDihedralAnglesHelix(helix_dict_list[ind]);
        logger.info('Dihedral angles generated sucessfully');

        try:
            helix_dict_list =writeHelixFile(helix_dict_list, pdb_code);
            plotRamachandranAngles(helix_dict_list,pdb_code);
            return helix_dict_list;

        except:
            logger.error('some error occured in plotting');
            return [];


    except:
        logger.error('Something Bad hapenned in dihedral analysis')
    

    return helix_dict_list;
    

### Master code to run all analysis

In [46]:
# specify PDF file and read it in text format
runBrokenHelix = 1; 
runGetDihedralAngles = 1; 
pdb_code_list = os.listdir('../PDB_files');
#pdb_code_list=['6sp7.pdb'];
for pdb_code in pdb_code_list:
    logger.info("Performing Analysis analysis for " +  pdb_code)
    pdb_file = open('../PDB_files/'+pdb_code, 'r');
    pdb_lines_complete = pdb_file.readlines();
    if(runBrokenHelix):
       helix_dict_list=  runBrokenHelixAnalysis(pdb_lines_complete,pdb_code);
    if(runGetDihedralAngles):
        helix_dict_list=getDihedralAngles(helix_dict_list,pdb_code);


Number of atoms records present in PDB are 53688
Number of  helix present in PDB are 168
Number of Sheets present in PDB are 0
Number of links present in PDB are 48
Number of atoms records present in PDB are 3477
Number of  helix present in PDB are 17
Number of Sheets present in PDB are 24
Number of links present in PDB are 29
